In [1]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pylab as plt
from IPython.display import display, HTML
from datetime import datetime
from pypfopt import risk_models
from pypfopt import expected_returns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from pypfopt import objective_functions

from backtest import BackTestStats, BaselineStats, BackTestPlot, backtest_strat, baseline_strat
from backtest import backtest_strat, baseline_strat

import env_portfolio
from env_portfolio import StockPortfolioEnv

import pandas as pd
import numpy as np

C:\Users\SANJANA\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
import config1
from config1 import config

In [3]:
config

<module 'config1.config' from 'C:\\Users\\SANJANA\\Desktop\\RL\\DOW_JONES_INTEGRATION\\config1\\config.py'>

In [4]:
import models
from models import DRLAgent


In [5]:
%store -r train_df
%store -r test_df
%store -r prices_full_train_df
%store -r prices_full_test_df
%store -r final_df
%store -r selected_stocks

In [6]:
prices_full_train_df.head()

,date,CMCSA,LIN,SO,KO,PFE,ACN,AMGN,VZ,MA,...,COST,USB,ABT,AXP,DHR,UPS,MCD,JNJ,TXN,V
0,2013-12-03,24.385000,124.239998,40.880001,40.349998,29.620493,77.089996,113.449997,49.599998,75.410004,...,123.820000,38.520000,37.599998,84.610001,55.928734,101.900002,96.379997,93.970001,42.840000,50.437500
1,2013-12-04,24.440001,123.620003,41.090000,40.369999,29.667933,76.070000,113.070000,49.369999,74.817001,...,122.970001,39.000000,37.360001,84.500000,55.845337,101.720001,95.709999,93.629997,42.430000,50.685001
2,2013-12-05,24.540001,122.489998,40.939999,39.830002,29.648956,75.790001,113.349998,48.910000,75.150002,...,120.949997,38.610001,36.919998,84.540001,55.830173,101.629997,95.430000,92.970001,42.660000,50.427502
3,2013-12-06,24.635000,125.459999,41.430000,40.459999,29.924099,75.529999,114.120003,49.480000,75.847000,...,122.059998,39.660000,37.529999,85.930000,56.952236,102.430000,96.800003,94.440002,43.490002,50.467499
4,2013-12-09,24.570000,125.410004,41.040001,40.400002,29.962049,75.129997,113.980003,49.570000,75.886002,...,121.660004,39.740002,37.200001,85.750000,57.149357,102.339996,95.720001,94.440002,43.580002,50.397499


In [7]:
train_df.shape

(36560, 12)

In [9]:
df = pd.DataFrame(prices_full_train_df)
df.set_index('date', inplace=True)
df

,CMCSA,LIN,SO,KO,PFE,ACN,AMGN,VZ,MA,DUK,COST,USB,ABT,AXP,DHR,UPS,MCD,JNJ,TXN,V
date,,,,,,,,,,,,,,,,,,,,
2013-12-03,24.385000,124.239998,40.880001,40.349998,29.620493,77.089996,113.449997,49.599998,75.410004,69.750000,123.820000,38.520000,37.599998,84.610001,55.928734,101.900002,96.379997,93.970001,42.840000,50.437500
2013-12-04,24.440001,123.620003,41.090000,40.369999,29.667933,76.070000,113.070000,49.369999,74.817001,69.919998,122.970001,39.000000,37.360001,84.500000,55.845337,101.720001,95.709999,93.629997,42.430000,50.685001
2013-12-05,24.540001,122.489998,40.939999,39.830002,29.648956,75.790001,113.349998,48.910000,75.150002,69.599998,120.949997,38.610001,36.919998,84.540001,55.830173,101.629997,95.430000,92.970001,42.660000,50.427502
2013-12-06,24.635000,125.459999,41.430000,40.459999,29.924099,75.529999,114.120003,49.480000,75.847000,70.190002,122.059998,39.660000,37.529999,85.930000,56.952236,102.430000,96.800003,94.440002,43.490002,50.467499
2013-12-09,24.570000,125.410004,41.040001,40.400002,29.962049,75.129997,113.980003,49.570000,75.886002,69.809998,121.660004,39.740002,37.200001,85.750000,57.149357,102.339996,95.720001,94.440002,43.580002,50.397499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-04,53.930000,245.419998,57.410000,49.939999,34.200001,247.000000,221.910004,54.799999,349.500000,86.459999,319.040009,51.020000,116.010002,142.679993,214.410004,159.449997,204.839996,153.070007,163.250000,211.500000
2021-03-05,55.090000,247.639999,58.349998,50.790001,34.389999,249.509995,227.729996,56.000000,360.880005,88.529999,317.320007,52.470001,117.250000,147.330002,218.350006,164.399994,207.369995,156.100006,167.940002,215.410004
2021-03-08,55.480000,253.679993,59.610001,51.639999,34.349998,249.389999,227.809998,56.790001,371.329987,90.220001,311.420013,54.049999,115.629997,150.270004,212.380005,160.869995,209.110001,157.399994,162.110001,220.270004


In [10]:
# Define a Function for Displaying the Cleaned Weights
def show_clean_p(port_df):
    p1_show_1 = (port_df.transpose()[0]).map(lambda x: "{:.3%}".format(x)).to_frame().transpose()
    return display(HTML(p1_show_1.to_html()))

In [11]:
# Plotting the Daily Draw Down

ticker_symb = ['VZ']
prices = df[ticker_symb]
window = 250

def get_daily_max_drawdown(prices, window):
    max_rolling = prices.rolling(min_periods=1, window=window).max()
    daily_drawdown = (prices / max_rolling) - 1
    max_daily_drawdown = daily_drawdown.rolling(min_periods=1, window=window).min()
    return daily_drawdown,max_daily_drawdown

max_rolling = prices.rolling(min_periods=1, window=window).max()

daily_drawdown, max_daily_drawdown = get_daily_max_drawdown(prices, window)
daily_drawdown.name = "{} daily drawdown".format(ticker_symb) 
#daily_drawdown = pd.DataFrame(daily_drawdown)

In [12]:
%matplotlib inline

# fig, ax = plt.subplots(figsize=(12,10))
# daily_drawdown.plot(ax=ax);
# ax.set_title("Daily Drawdown", fontsize=14)
# plt.legend(loc="best");
# plt.grid(True)
# plt.show()

In [23]:
totalIterations  = 10      # Main Stopping criteria = the total number of iterations
evalGoal         = 'risk'    # Minimze 'risk' or maximize 'sharpe'
minDesiredReturn = 0.075     # meet the minimum desired return. E.g. 7.5%
numPorfolios     = 5       # number of portfolios in swarm to simulate
method           = 'local'   # 'local' or 'global' best function name

MONTHS_IN_YEAR = 12 # The number of months in a year

# bounds for evaluation Function search space
lowerBound        = 0   # no weight in a stock
upperBound        = 1   # Max weight in stock

In [37]:
# Using the average daily return to calculate portfolio return

Returns = df.pct_change() # get the assets daily returns
# mean_daily_returns = Returns.mean().values

ExpectedReturns = Returns.mean().values

ExcessReturns = Returns.copy() - np.array(ExpectedReturns)

numPeriods = len(ExcessReturns.index)

# Covariances between stocks (variance covariance matrix)
VarCov = ExcessReturns.transpose().dot(ExcessReturns) / ( numPeriods - 1 )

In [38]:
# def evalExpectedReturn(prices, weigths):
#     months = (pd.to_datetime(df.index)[-1] - pd.to_datetime(df.index)[0]) / np.timedelta64(1, 'M')
#     months = np.floor(months)
#     total_return = (prices.iloc[-1].dot(weigths) - prices.iloc[0].dot(weigths)) / prices.iloc[0].dot(weigths)
#     annualized_return = ((1 + total_return) ** (12 / months)) - 1
#     return annualized_return

# def get_portfolio_variance(returns, weigths):
#     covariance_returns = returns.cov() * 250
#     var = np.dot(weigths.T, np.dot(covariance_returns, weigths))
#     return var

def evalExpectedReturn(MONTHS_IN_YEAR, Weights, ExpectedReturns):

    Weights = np.array(Weights)    

    expectedReturn = MONTHS_IN_YEAR * ( Weights.dot( ExpectedReturns.transpose() ) )
    return expectedReturn


# Function to Calculate the Risk
# -- Calculation: risk = sqrt(12) * sqrt( ( Weights * VarCov ) * t(Weights) )
# -----------------------------------------------------------------------------
def evalRisk(MONTHS_IN_YEAR, Weights, VarCov, ExpectedReturns, minDesiredReturn):
    
    # convert to numpy array
    Weights = np.array(Weights)
    
    # IF not the return desired then put a penalty on the portfolio
#     if evalExpectedReturn(MONTHS_IN_YEAR, Weights, ExpectedReturns) < minDesiredReturn:
    risk = 9999 # large penalty
        
    # # Else if desired return, then return actual risk
    # else:
#     risk = np.sqrt(MONTHS_IN_YEAR) * np.sqrt( ( Weights.dot(VarCov) ).dot( Weights.transpose() ) )
    
    return risk

In [39]:
rfr = 0.04 
#Risk free rate

In [40]:
def evalFunction(Weights, var):
    
    # For every portfolio portfolio, calculate the fitness value based on evaluation function
    
    ## Calculate the risk
    risk = evalRisk(MONTHS_IN_YEAR, Weights, VarCov, ExpectedReturns, minDesiredReturn)
    
    ## Calculate the sharpe ratio
    sharpeRatio = evalExpectedReturn(MONTHS_IN_YEAR, Weights, ExpectedReturns)

    fitnessValue = 0 # initialize the output variable

    # Evaluate using the risk (minimize the risk of portfolio)
    if evalGoal == 'risk':
        fitnessValue = risk
    
    # Else use the sharpe ratio (maximize the sharpe ratio of portfolio):
    # Note changed signs because we want to dynamically maximize
    else: 
        fitnessValue = -sharpeRatio

    return fitnessValue

# =============================================================================
# GLOBAL MIN VALUE AND POSITION SEARCH FUNCTION
# Returns the 2 element list (each containing a single value) with the global best portfolio's:
# ---- [0] min value and 
# ---- [1] associate Weights of 
# =============================================================================
def getGlobalBest(FitnessValues, Weights, numPorfolios):
    minValue = np.min(FitnessValues)         # Find the Minimum fitness value of all portfolios
    minIndex = FitnessValues.index(minValue) # Find the index of the Weights for the min. fit. value
    
    minPosition = Weights[minIndex][:] # Now get a copy of the portfolio's Weights with min index
    
    # Returns: the global best portfolio's minimum fitness value and its Weights
    return [minValue, minPosition] 


# =============================================================================
# LOCAL MIN VALUE AND POSITION SEARCH FUNCTION
# Topology: Ring structure with n neighbors  (default 2)
# Returns the 2 element list of lists with the each portfolio's local best within neighborhood
# ---- [0] min value and 
# ---- [1] associate Weights of 
# Can change numStocksInNbrhd to consider more or less in portfolio's neighborhood
# =============================================================================
def getLocalBest(FitnessValues, Weights, numPorfolios,
                  numStocksInNbrhd = 2):  # Number of portfolios to compare to for local best

    lBestFitValue = [] # will hold the best VALUE    of the n surrounding portfolios, for each portfolio
    lBestPosition = [] # will hold the best POSITION of the n surrounding portfolios, for each portfolio
    
    
    # For every portfolio in the portfolio swarm, (starting at n less than index 0)
    for portfolio in range(-numStocksInNbrhd, numPorfolios - numStocksInNbrhd):
        
        # Identify the two neighbors fitness value of this portfolio, 
        # which are the two precedng portfolios
        personalBestNeighbor1 = FitnessValues[portfolio]
        personalBestNeighbor2 = FitnessValues[portfolio + 1]
        
        # Identify the lowest fitness value of this portfolio's the two preciding neighbors
        minNeighValue = min(personalBestNeighbor1, personalBestNeighbor2)
        
        # Store the index of the portfolio
        minNeighIndex = FitnessValues.index(minNeighValue)
        
        # Store the portfolio's best neighbors fitness value and Weights
        lBestFitValue.append(FitnessValues[minNeighIndex])
        lBestPosition.append(Weights[minNeighIndex])
        
    # Returns a list of portfolios and the min of their n best fit. valued neighbors
    return[lBestFitValue, lBestPosition]


# If you needed to index the list just returned for global or local best
VALUE_IDX    = 0
POSITION_IDX = 1


# =============================================================================
# STEP 1 - SWARM INITIALIZATION / EVALUATION
# Randomly initialize a portfolio swarm instance
# Set the partical's best to it's starting Weights
# =============================================================================
def initializeSwarm(numPorfolios, numStocksInPort, functionToGetBest, evalGoal, absoluteMoveLimit):
    
    # In the current time period, Weights[portfolio] and Velocity[portfolio] of each portfolio i, 
    Weights  = [] # to hold the portfolio weights
    Velocity = [] # to hold the portfolio velocities

    # Lists containing info related to each portfolio in portfolio swarm
    pCurrFitValue = []  # X[portfolio] The current Weights of portfolio i
    
    # For each portfolio and stock, randomly initialize the...
    for portfolio in range(numPorfolios):
        # for stock in range(numStocksInPort):
            
        # Create a random feasible solution
        randomValues = np.random.rand(numStocksInPort)
        randomWeights = randomValues / sum(randomValues)
        
        # Ensure that it is an initial feasible solution meeting return contraint
        while evalExpectedReturn(MONTHS_IN_YEAR, randomWeights, ExpectedReturns) < minDesiredReturn:
            randomValues = np.random.rand(numStocksInPort)
            randomWeights = randomValues / sum(randomValues)
            
         
        # Position: give random solution of weights summing to 1
        Weights.append(randomWeights)
        
        # Velocity: give random value between lower and upper bound
        Velocity.append(np.random.rand(numStocksInPort) * absoluteMoveLimit)
    
        # 1.1 - Evaluate fitness value
        pCurrFitValue.append(np.sum(evalFunction(Weights[:], VarCov, ExpectedReturns, minDesiredReturn)))  # evaluate the current Weights's fitness value
        
    # 1.2 - Log the individual and global bests
    pBestPosition = Weights[:]       # initialize pBestPosition to the starting Weights
    pBestFitValue = pCurrFitValue[:]  # initialize pBestPosition to the starting Weights's value


    # 1.3 - Log the Global or local best (depends on chosen method) fitness value and Weights
    glBestFitValue, glBestPosition = functionToGetBest(pBestFitValue[:], pBestPosition[:], numPorfolios) 
    
    return [Weights, Velocity, pCurrFitValue, 
            pBestPosition, pBestFitValue, 
            glBestFitValue, glBestPosition]


# =============================================================================
# UPDATE VELOCITY AND POSITION 
# =============================================================================
def updateVelocityAndWeights(intertiaWeight, Velocity, Weights, phi1, phi2, pBestPosition, glBestPosition,
                              numPorfolios, numStocksInPort, absoluteMoveLimit):
# Velocity --------------------------------------------------------------------
    
    ## random weights of r for random Velocity adjustment
    r1, r2 = np.random.rand(), np.random.rand() 
    
    ## Calculations of updating Velocity, separated by 
    ## intertia + cognitive + social (for simplicity)
    vInertia   = np.multiply(intertiaWeight, Velocity[:])                          # Interia   component of updated Velocity
    vCognitive = np.multiply(phi1*r1, np.subtract( pBestPosition[:], Weights[:][portfolio])) # Cognitive component of ""
    vSocial    = np.multiply(phi2*r2, np.subtract(glBestPosition[:], Weights[:][portfolio])) # Social    component of ""
    
    ## Update the new Velocity to the summation of intertia, cognitive, and social
    newVelocity =  vInertia[:] + vCognitive[:] + vSocial[:]
    
    ## Limit the Velocity between the upper and lower bound limits
    for portfolio in range(numPorfolios):
        for stock in range(numStocksInPort):
        
            # If the new Velocity of portfolio i is > the ub move limit, then reduce to the limit
            if newVelocity[portfolio][stock] > absoluteMoveLimit:
                newVelocity[portfolio][stock] = absoluteMoveLimit
                
            # If the new Velocity of portfolio i is < the limit, then increase to the limit
            if newVelocity[portfolio][stock] < -absoluteMoveLimit:
                newVelocity[portfolio][stock] = -absoluteMoveLimit
        
    # Position ----------------------------------------------------------------
    
    ## Update new Weights based on the updated Velocity
    newWeights = Weights[:][portfolio] + newVelocity[:] 
    
    
    ## Make sure that the Weights is within the bounds -----------------------
    
    # For each portfolio and stock
    for portfolio in range(numPorfolios):
        for stock in range(numStocksInPort):
            
            # Push the new Weights to lower bound if lower
            if newWeights[portfolio][stock] < lowerBound:
                newWeights[portfolio][stock] = lowerBound
            
            # Push the new Weights to upper bound if higher
            if newWeights[portfolio][stock] > upperBound:
                newWeights[portfolio][stock] = upperBound

        
    for portfolio in range(numPorfolios):
            
        # Balance the weights to equal 1.
        # Be sure to keep move limit close to 1
        newWeights[portfolio] = np.array(newWeights[portfolio]) / sum(newWeights[portfolio])
        
    
    # Convert Weights and Velocity back to list ------------------------------
    newWeights  = newWeights.tolist()
    newVelocity = newVelocity.tolist()
    
    return [newWeights, newVelocity]


# =============================================================================
# Compare current Weights fitness value to the current best (for each portfolio)
# =============================================================================
def calculatePortfolioBests(Weights, numPorfolios, numStocksInPort,
                           pCurrFitValue, pBestPosition, pBestFitValue, evalGoal):
    # Calculate the fitness of the new Weights
    for portfolio in range(numPorfolios):
        # for stock in range(numStocksInPort):
            
        # Get the current fitness value of the new Weights
        pCurrFitValue[portfolio] = evalFunction(Weights[:][portfolio], VarCov, ExpectedReturns, minDesiredReturn)
        
        # Compare the current Weights' value to their person best
        if pCurrFitValue[portfolio] < pBestFitValue[portfolio]:
        
            # If better, then set the best VALUE to the current value (as a copy [:])
            pBestFitValue[portfolio] = pCurrFitValue[:][portfolio]
            
            # If better, then set the best POSITION to the current Weights  (as a copy [:])
            pBestPosition[portfolio] = Weights[:][portfolio]
    
    return [pCurrFitValue, pBestPosition, pBestFitValue]


# =============================================================================
# DISPLAY GLOBAL BEST AND DIMENSIONS FUNCTION
# Function for displaying the global best and its dimensions
# =============================================================================
def displayGlobalBest(glBestFitValue, glBestPosition, numStocksInPort, 
                      printDims, StockTickers, expectedReturn,
                      riskFreeRate, evalGoal, 
                      numPeriods=numPeriods,
                      MONTHS_IN_YEAR=MONTHS_IN_YEAR):
    
    # The sharpe ratio
    sharpeRatio = evalSharpeRatio(expectedReturn, riskFreeRate, glBestFitValue)

    # Total % return over the entire period
    monthsInSample = numPeriods/MONTHS_IN_YEAR # since on monthly basis
    returnOverPeriod = (1 + expectedReturn) ** monthsInSample
    
    # Flip signs if maximizing sharpe
    if evalGoal != 'risk':
        glBestFitValue = -glBestFitValue
        sharpeRatio    = -sharpeRatio
        
    # Print the global optima
    print('\n```')
    print('Key Summary Statistics -----------------')
    print(  'Global Best Annualized Risk:\t ' + str(round(100 * glBestFitValue, 1))  + '%')
    print(  'Annualized Expected Return:\t ' + str(round(100 * expectedReturn, 1))  + '%')
    print(  'Sharpe Ratio:\t\t\t% 0.2f' % sharpeRatio)
    print(  'Expected Return over', str(round(monthsInSample, 1) ), 'Years:  ' 
          + str(round(100 * returnOverPeriod, 1))  + '%')
    
    
    # Print each stock (if toggled)
    if printDims:
        print('\nGlobal best weights in each stock (Only includes stocks to invest in):')
        print('```')
        
        # Print the Weights of each stock in markdown table format
        print('| Ticker | Opt. Weight |', sep ='')
        print('|--------|-------------|')
        for stock in range(numStocksInPort):
            if glBestPosition[stock] > 0.001:
                print('|' + str(StockTickers[stock]).rjust(7, ' '), 
                      '|' + '{:.1f}'.format(100 * round(glBestPosition[stock], 3) ).rjust(11, ' ') + '% |'
                      )
    
                
# =============================================================================
# Plot the optimal weights and prices performance
# =============================================================================
def plotOptimalPerformance(RawStockDataPivot, OptimalWeights):
    
    # Calculate the weighted prices using adj close prices and optimal weights
    OptimizedPrices = RawStockDataPivot.copy().dropna().dot(OptimalWeights.copy())
    
    # Create a plot
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    
    # Plot the weighted prices
    plt.plot(OptimizedPrices)
    
    # Labels
    ax.set_title("Wt.'d Monthly Adj. Close Price of Optimal Portfolio")
    ax.set_xlabel('Month of Year (Year Only Shown for Simplicity)')
    ax.set_ylabel('Weighted Adj. Close Price (USD)')
    

In [41]:
%store -r 

In [42]:
selected_stocks

Index(['CMCSA', 'LIN', 'SO', 'KO', 'PFE', 'ACN', 'AMGN', 'VZ', 'MA', 'DUK',
       'COST', 'USB', 'ABT', 'AXP', 'DHR', 'UPS', 'MCD', 'JNJ', 'TXN', 'V'],
      dtype='object', name='stock_name')

In [43]:
df

,JNJ,PG,KO,MCD,USB,JPM,GD,VZ,MMM,PEP,CL,LIN,DHR,HON,LMT,MDT,V,RTX,T,COST
date,,,,,,,,,,,,,,,,,,,,
2013-12-03,93.970001,83.830002,40.349998,96.379997,38.520000,56.860001,90.339996,49.599998,126.599998,83.800003,65.370003,124.239998,55.928734,83.009888,138.770004,57.639999,50.437500,69.030838,26.238670,123.820000
2013-12-04,93.629997,83.349998,40.369999,95.709999,39.000000,57.189999,89.480003,49.369999,126.459999,82.650002,65.040001,123.620003,55.845337,82.781075,136.229996,56.930000,50.685001,68.558846,26.132931,122.970001
2013-12-05,92.970001,82.690002,39.830002,95.430000,38.610001,55.820000,89.230003,48.910000,126.830002,81.900002,64.540001,122.489998,55.830173,82.809677,136.660004,56.910000,50.427502,68.628067,25.868580,120.949997
2013-12-06,94.440002,84.519997,40.459999,96.800003,39.660000,56.060001,90.790001,49.480000,128.610001,83.150002,65.660004,125.459999,56.952236,84.201599,138.190002,58.139999,50.467499,69.930771,26.080059,122.059998
2013-12-09,94.440002,84.779999,40.400002,95.720001,39.740002,56.509998,90.529999,49.570000,128.570007,82.690002,65.690002,125.410004,57.149357,83.772583,138.929993,57.869999,50.397499,69.968536,26.200907,121.660004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-04,153.070007,122.150002,49.939999,204.839996,51.020000,150.559998,165.770004,54.799999,177.630005,128.830002,74.440002,245.419998,214.410004,202.940002,338.250000,115.269997,211.500000,74.419998,21.842899,319.040009
2021-03-05,156.100006,125.980003,50.790001,207.369995,52.470001,150.910004,170.520004,56.000000,180.800003,133.029999,76.059998,247.639999,218.350006,206.580002,340.429993,118.260002,215.410004,75.180000,22.371601,317.320007
2021-03-08,157.399994,127.309998,51.639999,209.110001,54.049999,152.910004,172.690002,56.790001,183.770004,132.130005,76.449997,253.679993,212.380005,207.699997,341.450012,116.669998,220.270004,75.610001,22.651056,311.420013


In [44]:
len(selected_stocks)

20

In [45]:
# =============================================================================
# SWARM OPTIMIZATION FUNCTION
# Parameters:
# ---- evalGoal:        Minimize 'risk' or maximize 'sharpe' ratio
# ---- numPorfolios:    The number of portfolios within the portfolio swarm
# ---- intertiaWeight:  The weight assigned to the intertia component of the Velocity eq.
# ---- phi1:            Cognitive weight of the Velocity equation. Note phi1 + phi2 <= 4
# ---- phi2:            Social    weight of the Velocity equation. Note phi1 + phi2 <= 4
# ---- totalIterations: The total number of iterations before stopping (Stopping criterion)
# ---- method:          Can use 'local' or 'global' best methods. 
# ----                  'local' uses Ring Structure with 2 neighbors by default 
# ----                  If using 2D and 'global' best method, will export CSV.
# ----                  Reason for exporting is to read in data to R for plotting iterations
# ---- printDims:       Print the dimensions' value or not
# ---- StockTickers:    Tickers of the stocks pulled
# ---- RawStockDataPivot: Raw stock adj.close prices pivoted for showing performance over duration
# =============================================================================

def optimizePortfolioPSO( # -------------------- Defaults --------------------
    evalGoal        = 'risk',   # Minimize 'risk' or maximize 'sharpe'
    minDesiredReturn= minDesiredReturn,# THe minimum desired return
    numPorfolios    = 5,        # number of portfolios in swar
    phi1            = 2,        # Cognitive weight
    phi2            = 2,        # Social weight
    intertiaWeight  = 0.1,      # Constant Inertia weighting value,
    absoluteMoveLimit = 0.1,    # max of 10 pp change in weight
    totalIterations = 10,    # Stopping criteria = the total number of iterations
    method          = 'local',  # 'local' or 'global' best function name
    printDims       = True,     # Print the stock's value or not,
    StockTickers    = selected_stocks, # Tickers of the stocks pulled,
    RawStockDataPivot = df # The raw stock data for plotting weighted performance
    ):
    
    numStocksInPort = len(selected_stocks) # number of dimensions of problem

    
    # Initialize to global best function by default
    functionToGetBest = getGlobalBest
    
    # If not using the global best, then switch to the local best method
    if method != 'global':
        functionToGetBest = getLocalBest    
    
    
    # -----------------------------------------------------------------------------
    # INITIALIZE POSITION AND VELOCITY, and INITIAL BESTS
    # the portfolio swarm will be represented as a list of Weights, velocities, values, 
    # pBestPosition, and pBestPosition values
    # Note: Weights[0] and Velocity[0] provides the Weights and Velocity of portfolio 0; 
    # Weights[1] and Velocity[1] provides the Weights and Velocity of portfolio 1; and so on.
    # -----------------------------------------------------------------------------
    
    
    # Step 1: Initialize portfolio swarm and get the portfolios' and global best (and current Weights)
    Weights, Velocity, pCurrFitValue, pBestPosition, pBestFitValue, glBestFitValue, glBestPosition = initializeSwarm(numPorfolios, numStocksInPort, 
                                                                                                                     functionToGetBest, evalGoal, absoluteMoveLimit)
    
    # Create empty lists for holding the portfolio swarm iterations
    positionIterations      = [] # Each portfolio's Velocity
    velocityIterations      = [] # Each portfolio's Weights
    gBestPositionIterations = [] # The current Global Best Position
    
    
    # -----------------------------------------------------------------------------
    # Main Loop 
    # -----------------------------------------------------------------------------
    
    iteration = 0
    
    # While meeting total iteration and min desired return requirements
    while (iteration < totalIterations) and ( ( evalExpectedReturn(MONTHS_IN_YEAR, glBestPosition[:], ExpectedReturns) < minDesiredReturn) or ( evalExpectedReturn(MONTHS_IN_YEAR, glBestPosition[:], ExpectedReturns) > minDesiredReturn * 1.01) ): 
        
        # Step 0: Keep track of each iterations/stock for Velocity, Weights, and current global best
        velocityIterations.append(Velocity)           
        positionIterations.append(Weight)           
        gBestPositionIterations.append(glBestPosition) 
        
        # Step 2: Update the Velocity and Weights
        Weights, Velocity = updateVelocityAndWeights(intertiaWeight, Velocity, Weights, 
                                                     phi1, phi2, pBestPosition, glBestPosition,
                                                     numPorfolios, numStocksInPort,
                                                     absoluteMoveLimit)
        
        # Step 3: Recalculate the portfolio and global bests
        pCurrFitValue, pBestPosition, pBestFitValue = calculatePortfolioBests(Weights, 
                                                                              numPorfolios, numStocksInPort,
                                                                              pCurrFitValue, 
                                                                              pBestPosition, 
                                                                              pBestFitValue, 
                                                                              evalGoal)
                
        # Step 4: Get the Global or local best (depends on chosen method) fitness value and Weights
        glBestFitValue, glBestPosition = functionToGetBest(pBestFitValue[:], pBestPosition[:], numPorfolios) 
        
        iteration += 1 # increment iterator

    
    # -----------------------------------------------------------------------------
    # Global Best
    # -----------------------------------------------------------------------------
    
    # Finally, if using the local best method, get the absolute best from the local bests
    if method == 'local':
        gBestFitValue, gBestPosition = getGlobalBest(glBestFitValue, glBestPosition, numPorfolios)
        expectedReturn = evalExpectedReturn(MONTHS_IN_YEAR, gBestPosition, ExpectedReturns)
    
    else: # if not local best, then change the gl best is the global best
        gBestFitValue, gBestPosition = glBestFitValue, glBestPosition
        expectedReturn = evalExpectedReturn(MONTHS_IN_YEAR, gBestPosition, ExpectedReturns)
        
    # -----------------------------------------------------------------------------
    # Print and Export
    # -----------------------------------------------------------------------------
    
    # Print the global (or local best) and each dimensions' Weights
    displayGlobalBest(gBestFitValue, gBestPosition, numStocksInPort, printDims, 
                      StockTickers, expectedReturn, riskFreeRate, evalGoal)
    
    # Show the weighted price perfgormance of the oprtimal portfolio weights
    plotOptimalPerformance(RawStockDataPivot, gBestPosition.copy())
    
    return [gBestPosition, gBestFitValue] # return the best weights and risk

In [47]:
# Call the function based on the imputs at the top
# gBestSolution = optimizePortfolioPSO(
#                     evalGoal        = evalGoal,        # Minimize 'risk' or maximize 'sharpe'
#                     minDesiredReturn= minDesiredReturn,# The minimum desired return
#                     numPorfolios    = numPorfolios,    # number of portfolios in swarm to simulate
#                     totalIterations = totalIterations, # Main Stopping criteria = the total number of iterations
#                     method          = method,          # 'local' or 'global' best function name
#                     StockTickers    = selected_stocks     # The stock tickers
#                     )
print("hey")
gBestSolution = optimizePortfolioPSO(
    evalGoal = evalGoal,
    minDesiredReturn = minDesiredReturn,
    numPorfolios = numPorfolios,
    totalIterations = totalIterations,
    method = method,
    StockTickers = selected_stocks,
    RawStockDataPivot = df
)


hey


KeyboardInterrupt: 

In [ ]:
stock_dimension = len(train_df.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

In [ ]:
weights_initial = [1/stock_dimension]*stock_dimension

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df = train_df.rename(columns={'Close': 'close'})
train_df = train_df.rename(columns={'Open': 'open'})
train_df = train_df.rename(columns={'High': 'high'})
train_df = train_df.rename(columns={'Low': 'low'})
train_df = train_df.rename(columns={'Volume': 'volume'})

In [ ]:
tech_indicator_list = ['f01','f02','f03','f04']

In [ ]:
env_kwargs = {
    "hmax": 500, 
    "initial_amount": 1000000, 
    "transaction_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": tech_indicator_list, 
    "action_space": stock_dimension, 
    "reward_scaling": 0,
    'initial_weights': [1/stock_dimension]*stock_dimension
}

In [ ]:
e_train_gym = StockPortfolioEnv(df = train_df, **env_kwargs)

In [ ]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

# A2C

In [ ]:
# initialize
agent = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

trained_a2c = agent.train_model(model=model_a2c, 
                                tb_log_name='a2c',
                                total_timesteps=50000)

In [ ]:
# A2C Train Model
e_trade_gym = StockPortfolioEnv(df = train_df, **env_kwargs)
env_trade, obs_trade = e_trade_gym.get_sb_env()

a2c_train_daily_return, a2c_train_weights = DRLAgent.DRL_prediction(model=trained_a2c,
                        test_data = train_df,
                        test_env = env_trade,
                        test_obs = obs_trade)

In [ ]:
test_df.head()

# PPO

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

# DDPG

In [ ]:
agent = DRLAgent(env = env_train)
DDPG_PARAMS = {"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001}


model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000)

# TD3

In [ ]:
agent = DRLAgent(env=env_train)
TD3_PARAMS = {"batch_size": 100, "buffer_size": 1000000, "learning_rate": 0.001}
model_td3 = agent.get_model("td3", model_kwargs=TD3_PARAMS)



In [ ]:
trained_td3 = agent.train_model(model=model_td3,
                                tb_log_name='td3',
                                total_timesteps=50000)